# AdaBoost

Codigo fuente: https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html#sphx-glr-auto-examples-ensemble-plot-adaboost-regression-py

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import make_scorer, roc_auc_score

In [2]:
import pandas as pd
import os

In [3]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [4]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

In [5]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [6]:
rng = np.random.RandomState(1)

In [7]:
%run $set_de_entrenamiento_testing_y_prediccion

In [8]:
training = labels_training.merge(df_features, how='inner', on='person')

In [9]:
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)

regr_1.fit(training.drop('label', axis=1), training['label'])
regr_2.fit(training.drop('label', axis=1), training['label'])


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x1a19e39c18>)

In [10]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

In [32]:
df_features

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,eventos noche_y,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,madrugada,maniana,tarde,noche
person,,,,,,,,,,,,,,,,,,,,,,,,,
4886f805,640.000000,360.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
ad93850f,640.000000,360.000000,0.000000,0.000000,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,...,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
0297fc1e,640.000000,360.000000,0.000000,0.000000,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,...,145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2d681dd8,768.000000,1360.000000,0.000000,0.000000,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,...,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
cccea85e,864.000000,1536.000000,0.000000,0.000000,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,...,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
4c8a8b93,640.000000,360.000000,0.000000,0.000000,14.0,8.0,2.0,0.0,14.0,0.0,13.0,9.0,...,67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1b9f7cf6,640.000000,360.000000,0.000000,0.000000,0.0,3.0,1.0,0.0,0.0,0.0,1.0,8.0,...,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
29ebb414,640.000000,360.000000,0.000000,0.000000,4.0,0.0,1.0,0.0,3.0,0.0,4.0,11.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
de8fe91b,800.000000,1280.000000,0.000000,0.000000,0.0,3.0,1.0,0.0,3.0,0.0,4.0,13.0,...,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [11]:
labels_test['label_predicted'] = regr_1.predict(labels_test.drop('label', axis=1))

In [12]:
%run $calcular_auc

auc score: 0.8423948414349782


In [13]:
labels_test['label_predicted'] = regr_2.predict(labels_test.drop(['label','label_predicted'], axis=1))

In [14]:
%run $calcular_auc

auc score: 0.8640616877490903


# Tuning an AdaBoost regressor

In [15]:
labels_with_features = labels.merge(df_features, how='inner', on='person')
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)

In [16]:
splits = 10 # cantidad de splits en el cross validation
n_iter_search = 20 # cantidad de combinaciones, en total splits*n_iter_search RF a probar

In [17]:
from sklearn.model_selection import RandomizedSearchCV


El mejor AdaBoost fue:

print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

# Seguir mejorando

print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

In [18]:
param_dist = {
 'n_estimators': list(range(10,400,10)),
 'learning_rate' : np.arange(0.01,0.2,0.04),
 'loss' : ['linear']
 }

pre_gs_inst = RandomizedSearchCV(regr_2,
 param_distributions = param_dist,
 cv=3,
 n_iter = 150,
 n_jobs=-1,scoring=make_scorer(roc_auc_score))

pre_gs_inst.fit(training.drop('label', axis=1), training['label'])

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x1a19e39c18>),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390], 'learning_rate': array([0.01, 0.05, 0.09, 0.13, 0.17]), 'loss': ['linear']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_

In [19]:
print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

score: 0.8693939541470019


{'learning_rate': 0.01, 'loss': 'linear', 'n_estimators': 360}

In [20]:
param_dist = {
 'n_estimators': [128,129,130,131,132],
 'learning_rate' : [0.075,0.08,0.085,0.09,0.095],
 'loss' : ['linear']
 }

pre_gs_inst = RandomizedSearchCV(regr_2,
 param_distributions = param_dist,
 cv=3,
 n_iter = 20,
 n_jobs=-1,scoring=make_scorer(roc_auc_score))

pre_gs_inst.fit(training.drop('label', axis=1), training['label'])

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x1a19e39c18>),
          fit_params=None, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'n_estimators': [128, 129, 130, 131, 132], 'learning_rate': [0.075, 0.08, 0.085, 0.09, 0.095], 'loss': ['linear']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=make_scorer(roc_auc_score),
          verbose=0)

In [21]:
print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

score: 0.8698277722232489


{'learning_rate': 0.075, 'loss': 'linear', 'n_estimators': 128}

## Predecir labels desconocidos

Vamos a predecir los nuevos labels.

In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
training = labels.merge(df_features, how='inner', on='person')

In [24]:
param = {
    'n_estimators': 124,
    'loss': 'linear',
    'learning_rate': 0.07,
    'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
}

In [25]:
regr_2 = AdaBoostRegressor(**param)

In [26]:
scores = cross_val_score(regr_2, training.drop('label', axis=1), training['label'], cv=10, scoring='roc_auc')

In [27]:
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.871227 (+/- 0.025514)


In [28]:
labels_to_predict_with_features = labels_to_predict.merge(df_features, how='inner', on='person')

In [29]:
assert(labels_to_predict.shape[0] == labels_to_predict_with_features.shape[0])

In [30]:
regr_2.fit(training.drop('label', axis=1), training['label'])

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=0.07, loss='linear', n_estimators=124,
         random_state=None)

In [31]:
labels_to_predict['label'] = regr_2.predict(labels_to_predict_with_features)

In [ ]:
# labels_to_predict.to_csv(predicciones_csv)